In [4]:
import pandas as pd
import plotly.express as px
import os

In [5]:
path = "data/pythia-70-benchmark/"

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
dfs = [pd.read_csv(os.path.join(path, f)) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)
df.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'act', 'batch_size', 'beta1', 'beta2', 'buffer_batches', 'buffer_mult',
       'buffer_size', 'cfg_file', 'd_in', 'data_path',
       'dead_direction_frequency', 'expansion_factor', 'have tried',
       'l1_coeff', 'l1_target', 'layer', 'lr', 'model', 'model_batch_size',
       'num_eval_batches', 'num_eval_prompts', 'num_eval_tokens',
       'num_training_tokens', 'reg', 'reg_coeff_2', 'save_checkpoint_models',
       'save_path', 'seed', 'seq_len', 'to try', 'tried', 'use_wandb', 'wd',
       'W_dec_norm', 'W_enc_norm', 'avg_directions', 'batch', 'bias_kurtosis',
       'bias_mean', 'bias_skew', 'bias_std', 'dead_directions', 'decoder_sim',
       'encoder_sim', 'entropy', 'epoch', 'feature_cosine_sim_mean',
       'feature_cosine_sim_variance', 'hoyer_loss', 'l1_loss', 'loss',
       'loss_recovered', 'mean_feature_unembed_cosine_sim_kurtosis',
       'mse_loss', 'reconstruction_loss', 'reg_loss',

In [14]:
# Update reg column with new names
df["reg"] = df["reg"].map({
    "l1": "L1",
    "pure_sqrt": "L0"
})

In [16]:
df[["layer", "reg", "loss_recovered", "avg_directions"]]

,layer,reg,loss_recovered,avg_directions
0,5,L0,0.927428,25.455315
1,5,L1,0.914093,25.590157
2,4,L0,0.810628,25.279528
3,4,L1,0.793476,25.045275
4,3,L0,0.832672,25.624607
5,3,L1,0.783144,25.660040
6,2,L0,0.841208,25.821260
7,2,L1,0.808678,24.539566
8,1,L0,0.899811,26.380905
9,1,L1,0.876527,26.567913


In [24]:
tmp_df = df[["layer", "reg", "loss_recovered", "avg_directions"]].copy()
tmp_df["loss_recovered"] = tmp_df["loss_recovered"]*100
# round all columns
tmp_df = tmp_df.round(2)
pivot_df = tmp_df.pivot(index='layer', columns='reg').reset_index()
pivot_df.columns = ['Layer', 'Loss Recovered (L0)', 'Loss Recovered (L1)', 'Avg Directions (L0)', 'Avg Directions (L1)']
# set layer to index
pivot_df = pivot_df.set_index('Layer')

tsv_output = pivot_df.to_csv(sep='\t')
print(tsv_output)

Layer	Loss Recovered (L0)	Loss Recovered (L1)	Avg Directions (L0)	Avg Directions (L1)
0	99.64	99.42	24.16	22.87
1	89.98	87.65	26.38	26.57
2	84.12	80.87	25.82	24.54
3	83.27	78.31	25.62	25.66
4	81.06	79.35	25.28	25.05
5	92.74	91.41	25.46	25.59



In [13]:
g = px.scatter(df, x="layer", y="loss_recovered", color="reg")
g.update_layout(
    title="Comparison L0 and L1 regularization on Pythia 70M",
    xaxis_title="Layer",
    yaxis_title="Loss recovered",
    legend_title_text='Regularization',
    width=800
)

new_names = {"pure_sqrt": "L0", "l1": "L1"}
g.for_each_trace(lambda t: t.update(name=new_names[t.name]))

In [8]:
px.scatter(df, x="layer", y="avg_directions", color="reg")